# Modelado
Aquí modelaremos

In [34]:
# dependencies
import numpy  as np
import pandas as pd
from   sklearn.impute import SimpleImputer
from   sklearn.preprocessing import OneHotEncoder, StandardScaler

In [2]:
# data
path = 'work/data/processed/celulares_procesasdos.csv'

In [3]:
# loading
df_inicio = pd.read_csv(path)

In [4]:
# conservo columnas a usar
columnas_ignorar = {'color', 'pantalla'}
df = df_inicio.drop(columns = columnas_ignorar).drop_duplicates().reset_index(drop = True)

In [5]:
df

,marca,producto_nombre,peso,camara_trasera,camara_frontal,procesador,ram,memoria,sistema_operativo,precio,tecnologia
0,samsung,galaxy z fold2,0.282,12,10,qualcomm,12,256,android,46799,5g
1,apple,iphone 11 pro max,0.272,12,12,apple,4,512,ios,33999,4glte
2,apple,iphone 11 pro,0.252,12,12,apple,4,512,ios,31279,4glte
3,samsung,galaxy note 20 ultra,0.302,12,10,qualcomm,8,256,android,30599,4g
4,samsung,galaxy z flip,0.183,12,10,qualcomm,8,256,android,29699,4glte
...,...,...,...,...,...,...,...,...,...,...,...
76,motorola,moto e6 plus,0.149,13,8,qualcomm,2,32,android,2990,4glte
77,motorola,moto e6 play,0.190,13,8,qualcomm,1,16,android,2499,4glte
78,motorola,moto e6 plus,0.149,13,8,mediatek,2,32,android,2990,4glte
79,huawei,honor 8a,0.176,13,8,mediatek,2,32,android,3299,4glte


In [7]:
# columnas categoricas
columnas_categoricas = ['marca', 'procesador', 'sistema_operativo', 'tecnologia']
# columnas numericas
columnas_numericas = ['peso', 'camara_trasera', 'camara_frontal', 'ram', 'memoria', 'precio']

### Llenado de valores faltantes
En ambos casos de usará el valor más frecuente: el promedio para las variables númericas, para las variables categoricas la moda.

In [39]:
# normalización de variables continuas
estandarizador = StandardScaler()
estandarizador.fit(df[columnas_numericas])
# estandarizador.transform(df[columnas_numericas])
imputador_numeric   = SimpleImputer(missing_values = np.nan, strategy = 'mean')
imputador_numeric.fit(df[columnas_numericas]).transform(df[columnas_numericas])
# df[columnas_numericas]
# imputador_categoric = SimpleImputer(mising_values = None, strategy = 'most_frequent')

array([[2.8200e-01, 1.2000e+01, 1.0000e+01, 1.2000e+01, 2.5600e+02,
        4.6799e+04],
       [2.7200e-01, 1.2000e+01, 1.2000e+01, 4.0000e+00, 5.1200e+02,
        3.3999e+04],
       [2.5200e-01, 1.2000e+01, 1.2000e+01, 4.0000e+00, 5.1200e+02,
        3.1279e+04],
       [3.0200e-01, 1.2000e+01, 1.0000e+01, 8.0000e+00, 2.5600e+02,
        3.0599e+04],
       [1.8300e-01, 1.2000e+01, 1.0000e+01, 8.0000e+00, 2.5600e+02,
        2.9699e+04],
       [2.7200e-01, 1.2000e+01, 1.2000e+01, 4.0000e+00, 2.5600e+02,
        2.8990e+04],
       [3.1200e-01, 1.2000e+01, 1.0000e+01, 8.0000e+00, 2.5600e+02,
        2.5990e+04],
       [2.5200e-01, 1.2000e+01, 1.2000e+01, 4.0000e+00, 2.5600e+02,
        2.5159e+04],
       [2.5200e-01, 1.2000e+01, 1.2000e+01, 4.0000e+00, 2.5600e+02,
        2.4990e+04],
       [2.5200e-01, 1.2000e+01, 1.2000e+01, 4.0000e+00, 2.5600e+02,
        2.4490e+04],
       [3.0800e-01, 1.2000e+01, 7.0000e+00, 4.0000e+00, 5.1200e+02,
        2.4303e+04],
       [3.1200e-01, 1

In [28]:
# codificación de variables categoricas
# one hot encoder
encoder = OneHotEncoder(handle_unknown = 'ignore', sparse = False)
encoder.fit(df[columnas_categoricas]).transform(df[columnas_categoricas])

array([[0., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 1., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       [0., 0., 1., ..., 0., 1., 0.]])

In [12]:
df[columnas_categoricas]

,marca,procesador,sistema_operativo,tecnologia
0,samsung,qualcomm,android,5g
1,apple,apple,ios,4glte
2,apple,apple,ios,4glte
3,samsung,qualcomm,android,4g
4,samsung,qualcomm,android,4glte
...,...,...,...,...
76,motorola,qualcomm,android,4glte
77,motorola,qualcomm,android,4glte
78,motorola,mediatek,android,4glte
79,huawei,mediatek,android,4glte
